In [72]:
import keras

from keras.models import Model
from keras.models import Sequential

from keras.layers.recurrent import LSTM

from keras.layers import Conv1D, MaxPool1D, Flatten, Input
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.utils import plot_model

from sklearn.model_selection import train_test_split

import numpy as np
import MySQLdb

In [2]:
conn = MySQLdb.connect(
    user='sequel',
    passwd='seQuel9_',
    host='localhost',
    db='coincheck'
)

c = conn.cursor()

In [186]:
query = '''
SELECT

ask.ask64,
ask.ask63,
ask.ask62,
ask.ask61,
ask.ask60,
ask.ask59,
ask.ask58,
ask.ask57,
ask.ask56,
ask.ask55,
ask.ask54,
ask.ask53,
ask.ask52,
ask.ask51,
ask.ask50,
ask.ask49,
ask.ask48,
ask.ask47,
ask.ask46,
ask.ask45,
ask.ask44,
ask.ask43,
ask.ask42,
ask.ask41,
ask.ask40,
ask.ask39,
ask.ask38,
ask.ask37,
ask.ask36,
ask.ask35,
ask.ask34,
ask.ask33,
ask.ask32,
ask.ask31,
ask.ask30,
ask.ask29,
ask.ask28,
ask.ask27,
ask.ask26,
ask.ask25,
ask.ask24,
ask.ask23,
ask.ask22,
ask.ask21,
ask.ask20,
ask.ask19,
ask.ask18,
ask.ask17,
ask.ask16,
ask.ask15,
ask.ask14,
ask.ask13,
ask.ask12,
ask.ask11,
ask.ask10,
ask.ask9,
ask.ask8,
ask.ask7,
ask.ask6,
ask.ask5,
ask.ask4,
ask.ask3,
ask.ask2,
ask.ask1,
ask.ask0,

bid.bid0,
bid.bid1,
bid.bid2,
bid.bid3,
bid.bid4,
bid.bid5,
bid.bid6,
bid.bid7,
bid.bid8,
bid.bid9,
bid.bid10,
bid.bid11,
bid.bid12,
bid.bid13,
bid.bid14,
bid.bid15,
bid.bid16,
bid.bid17,
bid.bid18,
bid.bid19,
bid.bid20,
bid.bid21,
bid.bid22,
bid.bid23,
bid.bid24,
bid.bid25,
bid.bid26,
bid.bid27,
bid.bid28,
bid.bid29,
bid.bid30,
bid.bid31,
bid.bid32,
bid.bid33,
bid.bid34,
bid.bid35,
bid.bid36,
bid.bid37,
bid.bid38,
bid.bid39,
bid.bid40,
bid.bid41,
bid.bid42,
bid.bid43,
bid.bid44,
bid.bid45,
bid.bid46,
bid.bid47,
bid.bid48,
bid.bid49,
bid.bid50,
bid.bid51,
bid.bid52,
bid.bid53,
bid.bid54,
bid.bid55,
bid.bid56,
bid.bid57,
bid.bid58,
bid.bid59,
bid.bid60,
bid.bid61,
bid.bid62,
bid.bid63,
bid.bid64

FROM array_asks AS ask
	LEFT JOIN array_bids AS bid
	ON ask.datetime = bid.datetime
	LEFT JOIN reg_data AS d
	ON ask.datetime = d.datetime

WHERE bid.datetime IS NOT NULL
	AND d.later_1m_class IS NOT NULL
    
ORDER BY ask.datetime 
'''
c.execute(query)
x = c.fetchall()

In [187]:
query = '''
SELECT
d.later_1m_class

FROM array_asks AS ask
	LEFT JOIN array_bids AS bid
	ON ask.datetime = bid.datetime
	LEFT JOIN reg_data AS d
	ON ask.datetime = d.datetime

WHERE bid.datetime IS NOT NULL
	AND d.later_1m_class IS NOT NULL
    
ORDER BY ask.datetime 
'''
c.execute(query)
y = c.fetchall()

In [225]:
X = np.array(x)
Y = np.array(y)
Y = np_utils.to_categorical(Y, 3)
print(X.shape, Y.shape)

(30207, 130) (30207, 3)


In [226]:
X = np.delete(X, range(0,17), 0)
Y = np.delete(Y, range(0,17), 0)
print(X.shape, Y.shape)

(30190, 130) (30190, 3)


In [232]:
last = X.shape[0] - 1
data_num = 1509
step = 20

indexes = [[0,1,2,3,4,5,6,7,8,last],
           [0,1,2,3,4,5,6,7,last-1, last],
           [0,1,2,3,4,5,6,last-2,last-1, last],
           [0,1,2,3,4,5,last-3,last-2,last-1, last],
           [0,1,2,3,4,last-4,last-3,last-2,last-1, last],
           [0,1,2,3,last-5,last-4,last-3,last-2,last-1, last],
           [0,1,2,last-6,last-5,last-4,last-3,last-2,last-1, last],
           [0,1,last-7,last-6,last-5,last-4,last-3,last-2,last-1, last],
           [0,last-8,last-7,last-6,last-5,last-4,last-3,last-2,last-1, last]]

reg_x = np.delete(X, [0,1,2,3,4,5,6,7,8,9], 0)
reg_y = np.delete(Y, [0,1,2,3,4,5,6,7,8,9], 0)

reg_x = step_x.reshape(data_num, step, 130)
reg_y = step_y.reshape(data_num, step, 3)


for index in indexes:
    step_x = np.delete(X, index, 0)
    step_y = np.delete(Y, index, 0)

    step_x = step_x.reshape(data_num, step, 130)
    step_y = step_y.reshape(data_num, step, 3)
    
    reg_x = np.concatenate([reg_x, step_x], 0)
    reg_y = np.concatenate([reg_y, step_y], 0)
print(reg_x.shape, reg_y.shape)

(15090, 20, 130) (15090, 20, 3)


In [233]:
train_X, test_X, train_Y, test_Y = train_test_split(reg_x, reg_y, train_size=0.8)
print(train_X.shape, test_X.shape, train_Y.shape, test_Y.shape)

(12072, 20, 130) (3018, 20, 130) (12072, 20, 3) (3018, 20, 3)


In [240]:
model = Sequential()
model.add(LSTM(64,
               batch_input_shape=(None, step, 130),
               unroll = True,
               implementation = 0,
               dropout = 0.5
              ))
model.add(Dense(3))
model.add(Activation("softmax"))
model.compile(loss="mean_squared_error", optimizer='adam', metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_25 (LSTM)               (None, 64)                49920     
_________________________________________________________________
dense_29 (Dense)             (None, 3)                 195       
_________________________________________________________________
activation_25 (Activation)   (None, 3)                 0         
Total params: 50,115
Trainable params: 50,115
Non-trainable params: 0
_________________________________________________________________


In [242]:
model.fit(train_X, train_Y[:, step-1], epochs = 3)

Epoch 1/3
12072/12072 [==============================] - 27s - loss: 0.2310 - acc: 0.3641    
Epoch 2/3
12072/12072 [==============================] - 19s - loss: 0.2204 - acc: 0.3955    
Epoch 3/3
12072/12072 [==============================] - 19s - loss: 0.2180 - acc: 0.4032    


In [243]:
loss, accuracy = model.evaluate(test_X, test_Y[:, step-1], verbose=0)
print("Accuracy = {:.2f}".format(accuracy))

Accuracy = 0.40
